## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [1]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

### Préparation des matrices de traits

In [2]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [64]:
rep="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Longitudinales/"
sample="Longitudinal-55-T1100000-F41134-X-"
fPMS="paradigmes.csv"
fPMO="Morphomes-paradigmes.csv"
fRulesPMS="Regles.pkl"
fRulesPMO="Morphomes-Regles.pkl"

### Préparation des cases du paradigme

In [65]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires
listeCases=casesTotales

# Préparation du calcul des analogies

### Calcul de la différence entre deux formes

In [66]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

### Accumulation des paires appartenant à un patron

In [67]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

### Transformation d'un patron en RegExp

In [68]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

### Substitution de sortie 
???

In [69]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

In [70]:
class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self):
        minLongueur=len(min(self.formes, key=len))
        maxLongueur=len(max(self.formes, key=len))
        if debug: print minLongueur, maxLongueur
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in self.formes])
            if debug: print phonemes
            if "." in phonemes:
                positions.append(".")
            else:
                positions.append("".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,case1,case2):
        self.patrons1={}
        self.patrons2={}
        self.case1=case1
        self.case2=case2

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
#        print forme1,forme2,patron
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
#        print patron12,patron21
        if not patron12 in self.patrons1:
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
            if debug: print "patron1", patron
            resultat1[patron]=self.patrons1[patron].calculerGM()
        resultat2={}
        for patron in self.patrons2:
            if debug: print "patron2", patron
            resultat2[patron]=self.patrons2[patron].calculerGM()
        return (resultat1,resultat2) 

# Classe pour la gestion des patrons, des classes et des transformations

In [71]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print "le numéro de forme n'est pas dans [1,2]",n

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=False):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
#                if debug: 
#                    print (forme, file=logfile)
#                    print ("pas de classe",idClasseForme, file=logfile)
#                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
#        else:
#            if debug:
#                print (forme, file=logfile) 
#                print ("pas de patron", file=logfile)
        return sortieForme
        

## Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [72]:
def OLDrapports(paradigme):
    (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    if len(paradigme)>0:
#        for index, row in paradigme.iterrows():
#            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

In [73]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

### Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [74]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [75]:
paradigmesPMS=pd.read_csv(rep+sample+fPMS,sep=";",encoding="utf8")
del paradigmesPMS[u"Unnamed: 0"]

In [76]:
with open(rep+sample+fRulesPMS, 'rb') as input:
    rulesPMS = pickle.load(input)
with open(rep+sample+fRulesPMO, 'rb') as input:
    rulesPMO = pickle.load(input)


In [77]:
def generateForms(lexeme,paradigmes):
    for forme in listeCases:
        if not pd.isnull(paradigmes[paradigmes["lexeme"]==lexeme][forme].iloc[0]):
            depart=paradigmes[paradigmes["lexeme"]==lexeme][forme].iloc[0]
            print forme,depart
            if depart!="nan":
                for case in listeCases:
                    print case,
                    if not isinstance(resultatsLecture[(forme, case)],str):
                        print resultatsLecture[(forme, case)].sortirForme(depart),
                print
                print

In [78]:
#generateForms(u"lever",paradigmes)

In [79]:
def showPatterns(listePaires,rules=rulesPMS):
    for resultat in listePaires:
        print "paire :",resultat
        for element in rules[resultat].patrons:
            print "\t"+element, rules[resultat].patrons[element]
        print

In [19]:
listePaires=[("inf","ppMP")]
showPatterns(listePaires,rules=rulesPMS)

paire : ('inf', 'ppMP')
	Etr-i ^(.*)mEtr$
	dr-zy ^kudr$
	rir-Er ^(.*)uvrir$
	- ^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJjlrwHE96Oêûô])e$
	u.ir-O. ^murir$
	ivr-Eky ^vivr$
	war-i ^aswar$
	r-y ^(.*[ptbdfsvzrE96O][jrwHiyEe926auOoêûâô][ptkbdg])r$
	avwar-y ^(.*)avwar$
	6vwar-y ^(.*[tdsz])6vwar$
	Etr-y ^(.*)kOnEtr$
	dr- ^(.*[ptkbdgjwH])êdr$
	âdr-i ^(.*)prâdr$
	Etr-e ^nEtr$
	war-y ^(.*[fsSvzZlr])war$
	r- ^(.*[ptkbdgfsSvzZmnJjlrwH][iEe])r$
	Erir-i ^(.*[E96aOêûâô])kErir$
	r-i ^(.*)sHivr$
	uvwar-y ^Emuvwar$
	ir-y ^(.*[ptbdfsvzmnlr])ir$



In [20]:
#paradigmes[paradigmes["lexeme"]==u"cuire"].stack().value_counts(dropna=True).sum()
#paradigmes[paradigmes["lexeme"]==u"dépendre"].stack()

In [80]:
"""
A dictionary difference calculator
Originally posted as:
http://stackoverflow.com/questions/1165352/fast-comparison-between-two-python-dictionary/1165552#1165552
"""


class DictDiffer(object):
    # TODO: on changed() avoid returning duplicate keys.
    # TODO:
    """
        Compares two dictionaries, traversing nested dictionaries,
        and provides four simple methods to access the differences between them:
            changed(), removed(), added(), unchanged()
         All four methods return a tuple of tuples of keys.
         When a given key is a nested one, it returns a tuple containing
         all the keys in the path to that key. For instance, given the sample above,
         the second "b" dict bellow contains the new "y" key under "x" and this in turn under "c",
         so it returns on tuple: ('c', ('x', 'y'))
    """

    def __init__(self, current_dict, past_dict):
        self._added,self._removed,self._changed,self._unchanged = self.diff_dicts(current_dict, past_dict)

        for key in (self._changed + self._unchanged):
#            print key, current_dict[key], past_dict[key]
            if isinstance(current_dict[key], dict) and isinstance(past_dict[key], dict):
                self._extend( key, self.__class__(current_dict[key], past_dict[key]) )

    def diff_dicts(self, current_dict, past_dict):
        """
        This is the original functionality for simple (not nested) dicts:
            Compares current_dict and past_dict and returns
            (added, removed, changed, unchanged) keys
        """
        set_current, set_past = set(current_dict.keys()), set(past_dict.keys())
        intersect = set_current.intersection(set_past)
        added = list(set_current - intersect)
        removed = list(set_past - intersect)
        changed = list(o for o in intersect if past_dict[o] != current_dict[o])
        unchanged = list(o for o in intersect if past_dict[o] == current_dict[o])
        return added, removed, changed, unchanged

    def _extend(self, parent, diff):
        """
            Adds the given differences to the appropriate members.
            Used with nested dicts.
        """
        self._added += [tuple([parent, o]) for o in diff._added]
        self._removed += [tuple([parent, o]) for o in diff._removed]
        self._changed += [tuple([parent, o]) for o in diff._changed]
        self._unchanged += [tuple([parent, o]) for o in diff._unchanged]

    def added(self):
        return tuple(self._added)
    def removed(self):
        return tuple(self._removed)
    def changed(self):
        return tuple(self._changed)
    def unchanged(self):
        return tuple(self._unchanged)


In [81]:
def showDifference(listePaires,rules1=rulesPMS,rules2=rulesPMO):
    for resultat in listePaires:
        print "paire :",resultat
    #    print resultatsLecture[resultat].patrons
    #    for element in resultatsLecture[resultat].patrons:
    #        print "\t"+element, resultatsLecture[resultat].patrons[element]
    #    print
        test=DictDiffer(rules1[resultat].patrons,rules2[resultat].patrons)
        if test.added(): print "added",test.added()
        if test.removed():
            print "missing"
            for element in test.removed():
                print "\t"+element,rules2[resultat].patrons[element]
            print "classe"
            print "\t"+element,rules2[resultat].classe
        if test.changed():
            print "overspecified"
            for element in test.changed():
                print "\t"+element,rules2[resultat].patrons[element]
                print "\t"+element,rules1[resultat].patrons[element]
                print
        if test.unchanged():
            print "optimal"
            print "\t"+element,rules2[resultat].patrons[element]

In [23]:
listePaires=[("inf","ppMP"),("pi1S","pi2S"),("pi2S","pi1P"),("pi2S","inf")]
showDifference(listePaires)

paire : ('inf', 'ppMP')
missing
	udr-Oly ^rEzudr$
	Er-y ^dEplEr$
	Or-o ^klOr$
	r-e ^Etr$
classe
	r-e {u'Etr-i': {u'Etr-i': 8}, u'Etr-e': {u'Etr-e': 1}, u'Etr-y': {u'Etr-y': 2}, u'war-i, war-y': {u'war-i': 1}, '-': {'-': 877}, u'6vwar-y, war-y': {u'6vwar-y': 7}, u'dr-': {u'dr-': 2}, u'\xe2dr-i, r-y': {u'\xe2dr-i': 7}, u'Etr-i, Etr-y': {u'Etr-i': 1}, u'Or-o': {u'Or-o': 1}, u'ir-y, r-, u.ir-O.': {u'u.ir-O.': 1}, u'ir-y, r-': {u'r-': 62, u'ir-y': 23}, u'ir-y, r-, Erir-i': {u'Erir-i': 2}, u'Etr-y, r-y': {u'Etr-y': 4}, u'r-y': {u'r-y': 31}, u'r-e': {u'r-e': 1}, u'ir-y, r-, rir-Er': {u'rir-Er': 6}, u'r-i': {u'r-i': 2}, u'dr-, r-y': {u'dr-': 5}, u'dr-zy': {u'dr-zy': 1}, u'ivr-Eky': {u'ivr-Eky': 2}, u'Er-y, r-': {u'Er-y': 1}, u'war-y': {u'war-y': 11}, u'uvwar-y, war-y': {u'uvwar-y': 2}, u'r-': {u'r-': 43}, u'r-y, udr-Oly': {u'udr-Oly': 1}, u'avwar-y, war-y': {u'avwar-y': 2}}
overspecified
	rir-Er ^(.*[uOo][fv])rir$
	rir-Er ^(.*)uvrir$

	ivr-Eky ^(.*)vivr$
	ivr-Eky ^vivr$

	Etr-y ^(.*[pk][E96aO]

In [82]:
cases=set([case for (case,autre) in rulesPMO.keys()])
systemes=[r+"-"+c for r in ["PMS","PMO"] for c in ["AC","SC"]]

In [83]:
def printSortie(paire,forme,rules,context=False):
    sorties=rules[paire].sortirForme(forme,contextFree=context)
    return [s+" : %2.2f"%sorties[s] for s in sorties]


In [84]:
paire=("fi2S","fi1P")
forme1=u"abâdOn6ra"

In [53]:
def printAlternatives(paire,forme1,lRules=systemes):
    if "PMS-AC" in lRules:
        print "PMS avec contexte\n\t",
        print "\n\t".join(printSortie(paire,forme1,rulesPMS,context=False))
    if "PMS-SC" in lRules:
        print "PMS sans contexte\n\t",
        print "\n\t".join(printSortie(paire,forme1,rulesPMS,context=True))
    if "PMO-AC" in lRules:
        print "PMO avec contexte\n\t",
        print "\n\t".join(printSortie(paire,forme1,rulesPMO,context=False))
    if "PMO-SC" in lRules:
        print "PMO sans contexte\n\t",
        print "\n\t".join(printSortie(paire,forme1,rulesPMO,context=True))

In [86]:
case1="ii1P"
forme1=u"mOrdjô"
paires=[(case1,c) for c in cases]
print forme1
for paire in paires:
    print paire
    printAlternatives(paire,forme1,lRules=["PMS-AC"])


mOrdjô
('ii1P', u'ii1P')
PMS avec contexte
	mOrdjô : 1.00
('ii1P', u'pP')
PMS avec contexte
	mOrdâ : 1.00
('ii1P', u'is1S')
PMS avec contexte
	
('ii1P', u'ii1S')
PMS avec contexte
	mOrdE : 1.00
('ii1P', u'ppMP')
PMS avec contexte
	mOrdi : 0.33
	mOrdy : 0.33
	mOrde : 0.33
('ii1P', u'ai1P')
PMS avec contexte
	mOrdim : 0.34
	mOrdam : 0.66
('ii1P', u'ai1S')
PMS avec contexte
	mOrdi : 0.18
	mOrdE : 0.82
('ii1P', u'pc1S')
PMS avec contexte
	mOrdirE : 0.24
	mOrd6rE : 0.52
	mOrdrE : 0.24
('ii1P', u'pi2S')
PMS avec contexte
	mOr : 0.91
	mOrd : 0.09
('ii1P', u'inf')
PMS avec contexte
	mOrdr : 0.14
	mOrde : 0.80
	mOrdir : 0.07
('ii1P', u'pi2P')
PMS avec contexte
	mOrde : 1.00
('ii1P', u'ppFP')
PMS avec contexte
	mOrdi : 0.29
	mOrde : 0.71
('ii1P', u'ps1P')
PMS avec contexte
	
('ii1P', u'is2P')
PMS avec contexte
	
('ii1P', u'ps1S')
PMS avec contexte
	mOrd : 1.00
('ii1P', u'pI2S')
PMS avec contexte
	mOrd : 1.00
('ii1P', u'fi2P')
PMS avec contexte
	mOrdire : 0.22
	mOrd6re : 0.48
	mOrdre : 0.30
('ii1